Ejercicio: Transformaciones avanzadas en dataset de e-commerce

Datos base:

In [3]:
import pandas as pd
import numpy as np

# Clientes
clientes = pd.DataFrame({
    'cliente_id': range(1, 6),
    'nombre': ['Ana', 'Juan', 'María', 'Pedro', 'Laura'],
    'segmento': ['Premium', 'Regular', 'Premium', 'Regular', 'VIP']
})

# Pedidos
pedidos = pd.DataFrame({
    'pedido_id': range(1, 11),
    'cliente_id': np.random.choice(range(1, 6), 10),
    'producto': np.random.choice(['A', 'B', 'C', 'D'], 10),
    'precio': np.random.uniform(50, 500, 10).round(2),
    'fecha': pd.date_range('2024-01-01', periods=10)
})

print("Clientes y pedidos cargados")

Clientes y pedidos cargados


Enriquecer datos con joins:

In [5]:
# Unir pedidos con información de clientes
pedidos_enriquecidos = pd.merge(
    pedidos, 
    clientes, 
    on='cliente_id', 
    how='left'
)

print("Pedidos con información de clientes:")
print(pedidos_enriquecidos.head())

Pedidos con información de clientes:
   pedido_id  cliente_id producto  precio      fecha nombre segmento
0          1           3        B  145.23 2024-01-01  María  Premium
1          2           4        D   82.01 2024-01-02  Pedro  Regular
2          3           5        C  234.56 2024-01-03  Laura      VIP
3          4           3        D  257.00 2024-01-04  María  Premium
4          5           1        C  198.80 2024-01-05    Ana  Premium


Calcular métricas derivadas:

In [6]:
# Calcular métricas por cliente
metricas_cliente = pedidos_enriquecidos.groupby(['cliente_id', 'nombre', 'segmento']).agg({
    'pedido_id': 'count',
    'precio': ['sum', 'mean', 'max'],
    'fecha': 'max'  # Última compra
}).round(2)

# Aplanar columnas multi-nivel
metricas_cliente.columns = ['num_pedidos', 'total_gastado', 'gasto_promedio', 'gasto_maximo', 'ultima_compra']
metricas_cliente = metricas_cliente.reset_index()

print("\nMétricas por cliente:")
print(metricas_cliente)


Métricas por cliente:
   cliente_id nombre segmento  num_pedidos  total_gastado  gasto_promedio  \
0           1    Ana  Premium            3         773.86          257.95   
1           3  María  Premium            3         507.94          169.31   
2           4  Pedro  Regular            2         445.71          222.86   
3           5  Laura      VIP            2         647.44          323.72   

   gasto_maximo ultima_compra  
0        459.51    2024-01-10  
1        257.00    2024-01-09  
2        363.70    2024-01-06  
3        412.88    2024-01-07  


Validar reglas de negocio:

In [7]:
def validar_reglas_negocio(df):
    validaciones = []
    
    # VIP deben tener al menos 2 pedidos
    vip_insuficientes = df[(df['segmento'] == 'VIP') & (df['num_pedidos'] < 2)]
    if len(vip_insuficientes) > 0:
        validaciones.append(f"VIPs con pocos pedidos: {len(vip_insuficientes)}")
    
    # Premium no deben exceder gasto máximo
    premium_excesivos = df[(df['segmento'] == 'Premium') & (df['gasto_maximo'] > 800)]
    if len(premium_excesivos) > 0:
        validaciones.append(f"Premiums con gastos excesivos: {len(premium_excesivos)}")
    
    return validaciones

reglas_incumplidas = validar_reglas_negocio(metricas_cliente)
print(f"\nReglas de negocio incumplidas: {reglas_incumplidas}")


Reglas de negocio incumplidas: []


Verificación: ¿Qué tipo de join usarías cuando quieres mantener todos los registros de una tabla principal? ¿Cómo decides qué métricas calcular para un análisis específico?

Requerimientos:
Pandas para manipulación de datos
Comprensión de operaciones de conjunto
Conocimiento de reglas de negocio

1) ¿Qué tipo de join usarías para mantener todos los registros de una tabla principal?
Respuesta:
Se utiliza un LEFT JOIN.
Este join conserva todas las filas de la tabla principal (la de la izquierda) y agrega la información de la tabla secundaria cuando existe coincidencia por la clave. Si no hay coincidencia, los campos de la tabla secundaria quedan como NaN.
En el ejercicio, pedidos es la tabla principal porque el análisis se centra en las transacciones, y clientes aporta atributos descriptivos.

2) ¿Cómo decides qué métricas calcular para un análisis específico?
Respuesta:
Las métricas se definen según:
- Objetivo del análisis (comportamiento del cliente, ingresos, frecuencia).
- Reglas de negocio (segmentación, límites, condiciones mínimas).
- Nivel de agregación requerido (por cliente, por segmento, por período).

En este caso:
- count(pedido_id) → frecuencia de compra.
- sum(precio) → valor total generado.
- mean(precio) → gasto promedio.
- max(precio) → control de extremos (reglas).
- max(fecha) → recencia (última compra).


Conclusión:
LEFT JOIN asegura integridad del dataset principal y las métricas se seleccionan en función de decisiones de negocio, no solo de disponibilidad de datos.